Gdrive

In [ ]:
gdrive_project_pathname = '/My Drive/w210'  #@param {type: "string"}

In [ ]:
import os.path
from google.colab import drive

# Mount the current user's Google Drive
GOOGLE_DRIVE_MOUNT_POINT = "/content/drive"
print("Mounting Google Drive beneath %s" % GOOGLE_DRIVE_MOUNT_POINT)
drive.mount(GOOGLE_DRIVE_MOUNT_POINT, force_remount = True)

# Build the pathname to the project's folder residing beneath the current user's Google Drive
if not gdrive_project_pathname.startswith("/"):
  gdrive_project_pathname = "/" + gdrive_project_pathname
abs_project_pathname = GOOGLE_DRIVE_MOUNT_POINT + gdrive_project_pathname
print("Project folder: %s" % gdrive_project_pathname)
data_project_pathname = abs_project_pathname + '/data/'

GEE

In [1]:
!pip3 install earthengine-api 

In [2]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [5]:
import ee
import geopandas
import numpy as np
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box
import warnings
from functools import reduce
from math import sin, cos, sqrt, atan2, radians

In [44]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

Create a dictionary of countries with its boundary coordinates

In [7]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
country_polygon_dict = dict()
for index, row in world.iterrows():
    if 'MultiPolygon' in str(type(row['geometry'])):
        i = 0
        for poly in row['geometry']:
            # Remove small area that's not needed for sampling
            if poly.area > 5:
                # Area is in the unit of squre degrees
                country_polygon_dict[row['iso_a3']+'_'+str(i)] = (poly, round(poly.area*0.2))
                i +=1 
    # Here implies a Polygon
    elif row['geometry'].area > 5:
        country_polygon_dict[row['iso_a3']] = (row['geometry'], round(row['geometry'].area*0.2))

In [ ]:
world[world.iso_a3 == 'ARG'].head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
9,44293293,South America,Argentina,ARG,879400.0,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [ ]:
# Remove miscellaneous countries
del country_polygon_dict['-99']
del country_polygon_dict['-99_0']
del country_polygon_dict['-99_1']
del country_polygon_dict['-99_2']

In [ ]:
country_polygon_dict

In [73]:
def country_bounds(area_of_interest_shapely, country_code, edge_len):

    return_list_coords = []
    min_lon, min_lat, max_lon, max_lat = area_of_interest_shapely.bounds
    dict_coords = {}
    point_list = []
    lat_list = []
    lon_list = []
    counter = 0
    center_lon = min_lon
    center_lat = max_lat
    # print(center_lat,min_lat,  center_lon, max_lon)
    while center_lon <= max_lon + edge_len/2:
        while center_lat >= min_lat - edge_len/2:
            # print(center_lat, center_lon)
            point_list.append(counter)         
            lat_list.append(center_lat)
            lon_list.append(center_lon)
        
            counter +=1
            center_lat -= edge_len
        
        center_lon += edge_len
        center_lat = max_lat
        
    dict_coords['POINTS'] = point_list
    dict_coords['Latitude'] = lat_list
    dict_coords['Longitude'] = lon_list
#     print("Total coords: ", counter)
    df = pd.DataFrame(dict_coords)
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude).buffer(edge_len - edge_len/2))
    gdf.crs = "EPSG:4326"
    point_within_country = geopandas.sjoin(gdf, world[world.iso_a3 == country_code], how="inner", op='intersects')
    for index, row in point_within_country.iterrows():
      (lat, lon) = (row['Latitude'], row['Longitude'])
      # (center_lat, center_lon) = (point_within_arg['Latitude'],	point_within_arg['Longitude'])
      return_list_coords.append((lat, lon))
    
    return return_list_coords

In [7]:
test_arg = country_bounds(country_polygon_dict['ARG_0'][0], 'ARG', 0.1)

In [ ]:
# https://gis.stackexchange.com/questions/6412/generate-points-that-lie-inside-polygon
    
    
# https://stackoverflow.com/questions/58802921/using-geopandas-how-to-randomly-select-in-each-polygon-5-points-by-sampling-met
    
    
# https://developers.google.com/earth-engine/geometries

Pull Sentinel-2 Data

In [69]:
#######old code - changes

def satellite_imagery_Sentinel(area_of_interest_shapely, country_code, edge_len,
                      start_date, end_date, 
                      plot_option, resolution=30):
    
    area_of_interest_ee = ee.Geometry.Polygon(list(area_of_interest_shapely.exterior.coords))

    
    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
                 .filter(ee.Filter.gt('system:asset_size', 800000000))
                 .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50))
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    print("Total number of images in the collection: ", img_collect.size().getInfo())
        
    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        warnings.warn('Multiple tiles are selected. Proceed with caution.')
        print('Number of tiles selected: ', len(unique_tiles))
        
    # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
    band_blue = 'B2' #10m
    band_green = 'B3' #10m
    band_red = "B4"  #10m
    band_nir = 'B8'  #10m
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
    def calc_SAVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        savi =  ee.Image(img.expression(
            '(1 + L) * float(nir - red)/ (nir + red + L)',
            {
                'nir': img.select(band_nir),
                'red': img.select(band_red),
                'L': 0.5
            })).rename(["savi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(savi)
        return composite

    # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
    #     C1=6, C2=7.5, and L=1
    def calc_EVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        evi = ee.Image(img.expression(
          '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
          {   
              'nir': img.select(band_nir),
              'red': img.select(band_red),
              'blue': img.select(band_blue),
              'L': 0.2,
              'C1': 6,
              'C2': 7.5
          })).rename(["evi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(evi)
        return composite
                   
    def add_landcover(img):
        landcover = ee.Image("USGS/GFSAD1000_V1")
        composite = img.addBands(landcover)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.median()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                        data = ee_object.getInfo(),
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                        tiles = map_id_dict['tile_fetcher'].url_format,
                        attr = 'Google Earth Engine',
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    print(unique_month)
    
    if len(unique_month) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    
    
    img_calc_month_dict = dict()
    temp_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month))
        
    # Create a folium map object.
    center_lon, center_lat = list(area_of_interest_shapely.centroid.coords)[0]
    myMap = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    # Add the box around the area of interest
    folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)

    ###adding boxes
    edge_len = edge_len #4.0 # 0.5
    list_coords = country_bounds(area_of_interest_shapely, country_code, edge_len)
    print("total sub rects: ", len(list_coords))
    for coord in list_coords:
        center_lat, center_lon = coord[0], coord[1]
#         folium.Marker(location=[center_lat, center_lon]).add_to(myMap)
        rect_of_interest_shapely = box(center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2)
        # Add the box around the area of interest
        folium.GeoJson(rect_of_interest_shapely).add_to(myMap)

    visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
    if plot_option == 'NDVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'SAVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'EVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=plot_option+' '+month)

    # Add a layer control panel to the map.
    myMap.add_child(folium.LayerControl())

#     return myMap, out_df
    return myMap

In [75]:
satellite_imagery_Sentinel(area_of_interest_shapely=country_polygon_dict['IND'][0], country_code='IND', edge_len = 4.0,
                  start_date='2018-1-01', end_date='2018-1-31', plot_option='NDVI')

Total number of images in the collection:  1700


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Multiple tiles are selected. Proceed with caution.


Number of tiles selected:  406
['201801']
total sub rects:  34


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:130: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [ ]:
India_1_lat, India_1_lon = 23.967052, 72.400000
India_1_edge_len = 0.005

In [ ]:
satellite_imagery_Sentinel(area_of_interest_shapely=country_polygon_dict['IND'][0], country_code='IND',
                  start_date='2018-1-01', end_date='2018-1-31', plot_option='NDVI')

In [52]:
satellite_imagery_Sentinel(area_of_interest_shapely=country_polygon_dict['ARG_0'][0], country_code='ARG',
                  start_date='2018-1-01', end_date='2018-3-31', plot_option='NDVI')

Total number of images in the collection:  4893


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Multiple tiles are selected. Proceed with caution.


Number of tiles selected:  375
['201801', '201802', '201803']
Total coords:  48
size lat_list:  48
total sub rects:  48


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:130: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [ ]:
min_lon, min_lat, max_lon, max_lat = country_polygon_dict['ARG_0'][0].bounds

NameError: ignored

In [ ]:
type(country_polygon_dict['ARG_0'][0])

shapely.geometry.polygon.Polygon

In [ ]:
min_lon, min_lat, max_lon, max_lat

(-73.4154357571201,
 -52.34998340612768,
 -53.628348965048744,
 -21.83231047942072)